In [0]:
%run "../includes/configurations"

Join Races and Circuits data

In [0]:
from pyspark.sql.functions import col

In [0]:
races_df=spark.read.parquet(f"{processed_folder_path}/races").select("race_id","race_year","race_timestamp",col("name").alias("race_name"),"circuit_id").withColumn("race_date", col("race_timestamp").cast("date")).drop("race_timestamp")

In [0]:
circuit_df=spark.read.parquet(f"{processed_folder_path}/circuits").select("circuit_id",col("location").alias("circuit_location"))

In [0]:
drivers_df=spark.read.parquet(f"{processed_folder_path}/drivers").select("driver_id",col("number").alias("driver_number"),col("name").alias("driver_name"),col("nationality").alias("driver_nationality"))

In [0]:
results_df=spark.read.parquet(f"{processed_folder_path}/results").select("grid","race_id","driver_id",col("constructor_id").alias("constructorid"),"fastest_lap",col("time").alias("race_time"),"points")

In [0]:
race_circuit_df=races_df.join(circuit_df,on="circuit_id").select("race_id","race_year","race_date","race_name","circuit_location")

In [0]:
qualifying_df=spark.read.parquet(f"{processed_folder_path}/qualifying").select("race_id","driver_id","constructorid")

In [0]:
constructor_df=spark.read.parquet(f"{processed_folder_path}/constructors").select(col("constructor_id").alias("constructorid"),col("constructor_ref").alias("team"))

In [0]:
display(constructor_df)

In [0]:
race_circuit_qualifying_df=race_circuit_df.join(qualifying_df,on="race_id").select("race_id","race_year","race_date","race_name","circuit_location","driver_id","constructorid")

In [0]:
display(race_circuit_qualifying_df)

In [0]:
display(race_circuit_qualifying_df)

In [0]:
race_circuit_qualifying_driver_df=race_circuit_qualifying_df.join(drivers_df,on="driver_id").select("race_id","race_year","race_date","race_name","circuit_location","driver_id","driver_number","driver_nationality","driver_name","constructorid")

In [0]:
display(race_circuit_qualifying_driver_df)

In [0]:
race_circuit_qualifying_constructor_df=race_circuit_qualifying_driver_df.join(constructor_df,on="constructorid").select("race_id","race_year","race_date","race_name","circuit_location","driver_id","team","constructorid","driver_number","driver_nationality","driver_name")

In [0]:
display(race_circuit_qualifying_constructor_df)

In [0]:
from pyspark.sql.functions import current_date

In [0]:
presentation_layer_df=race_circuit_qualifying_constructor_df.join(results_df,on=["race_id","driver_id","constructorid"]).select("race_year","race_name","race_date","circuit_location","driver_name","driver_number","driver_nationality","team","fastest_lap","race_time","points").withColumn("created_date",current_date())

In [0]:
display(presentation_layer_df)

In [0]:
display(presentation_layer_df.filter((presentation_layer_df.race_year==2020) & (presentation_layer_df.race_name=="Abu Dhabi Grand Prix")).orderBy(presentation_layer_df.points.desc()))

In [0]:
presentation_layer_df.write.mode("overwrite").parquet("/mnt/formula1jitdl/presentation")

In [0]:
display(spark.read.parquet("/mnt/formula1jitdl/presentation"))